In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('retina')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from openpyxl import load_workbook

import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox
from IPython.display import HTML
import base64

#import os
from email.message import EmailMessage
import ssl
import smtplib

pd.set_option('display.max_rows', None)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def init(df):
    df['prior_matches'] = [ [] for _ in range(len(df)) ]
    df['current_match'] = [ [] for _ in range(len(df)) ]
    df['current_group'] = -1
    df['num_prior_matches'] = 0
    df['size_prev_match'] = 0 # whether were in a 3 or 4 group previously, or 0 if new

def complete_match():
    # move the current match to the end of the list of prior matches
    for index, row in data.iterrows():
        data.loc[index, 'prior_matches'].extend(row['current_match'])
        data.loc[index, 'num_prior_matches'] = len(row['prior_matches'])
    data['size_prev_match'] = [len(a) for a in data['current_match'].tolist()]
    data['current_match'] = [ [] for _ in range(len(data)) ]
    data['current_group'] = -1

def possible_matches():
    # after each match, run this to determine future possible matches
    data['possible_matches'] = [ [] for _ in range(len(data)) ]
    data['num_possible_matches'] = -1
    for index, row in data.iterrows():
        non_previous_matches = data.index.to_numpy()[~np.in1d(data.index.to_numpy(), row['prior_matches'])]
        non_same_department = data.index.to_numpy()[~(row['department'] == data['department'])]
        data.loc[index, 'possible_matches'].extend(np.intersect1d(non_previous_matches, non_same_department))
        data.loc[index, 'num_possible_matches'] = len(data.loc[index, 'possible_matches'])

def perform_random_loop(df):
    # variable to determine when we've succeeded
    out = 0
    # clear any attemps to match that failed
    df['current_match'] = [ [] for _ in range(len(df)) ]
    df['current_group'] = -1
    # create a random column 
    df.loc[:, 'randint'] = np.random.choice(np.arange(0, len(df)), size=len(df), replace=False)
    
    groupnum = 1 # a counter for the group number
    # iterate through, starting with the most number of possible matches 
    for i, (index, row) in enumerate(df.sort_values(['size_prev_match', \
                                                     'num_possible_matches', \
                                                     'randint']).iterrows()):
        # select possible matches for person1
        if i == 0:
            p1_possible = df.loc[index, 'possible_matches']
        elif i > 0 :
            if len(remaining) == 0:
                out = 1
                return out
                break
            elif index not in remaining.index.tolist(): 
                continue
            else:
                p1_possible = np.intersect1d(remaining.loc[index, 'possible_matches'], \
                                             remaining.index.tolist())
        if len(p1_possible) <= 1:
            return out
            break
        # pick a random person2
        p2 = df.loc[p1_possible].sample(1)
        p2_possible = p2['possible_matches'].tolist()
        # take person1 possible matches and remove person2 and all of person2's not possible matches
        p1p2_possible_step1 = np.array(p1_possible)[~np.isin(p1_possible, p2.index.tolist())] # remove p2
        p1p2_possible = p1p2_possible_step1[np.isin(p1p2_possible_step1, p2_possible)]

        if len(p1p2_possible) == 0:
            return out
            break
        # pick a random person3
        p3 = df.loc[p1p2_possible].sample(1)
        p3_possible = p3['possible_matches'].tolist()

        if i < number4groups*4:
            # take person3 out oc p1p2_possible
            p1p2p3_possible_step1 = np.array(p1p2_possible)[~np.isin(p1p2_possible, p3.index.tolist())] 
            # keep only person3's possible matches 
            p1p2p3_possible = p1p2p3_possible_step1[np.isin(p1p2p3_possible_step1, p3_possible)]
            
            if len(p1p2p3_possible) == 0:
                return out
                break
            # pick a random person4
            p4 = df.loc[p1p2p3_possible].sample(1)

            # write the current match for all *4* group members
            df.loc[index, 'current_match'].extend([index, p2.index[0], p3.index[0], p4.index[0]])
            df.loc[p2.index[0], 'current_match'].extend([p2.index[0], index, p3.index[0], p4.index[0]])
            df.loc[p3.index[0], 'current_match'].extend([p3.index[0], index, p2.index[0], p4.index[0]])
            df.loc[p4.index[0], 'current_match'].extend([p4.index[0], index, p2.index[0], p3.index[0]])
            
            df.loc[index, 'current_group'] = groupnum
            df.loc[p2.index[0], 'current_group'] = groupnum
            df.loc[p3.index[0], 'current_group'] = groupnum
            df.loc[p4.index[0], 'current_group'] = groupnum
            
        else:
            # write the current match for all *3* group members
            df.loc[index, 'current_match'].extend([index, p2.index[0], p3.index[0]])
            df.loc[p2.index[0], 'current_match'].extend([p2.index[0], index, p3.index[0]])
            df.loc[p3.index[0], 'current_match'].extend([p3.index[0], index, p2.index[0]])
            
            df.loc[index, 'current_group'] = groupnum
            df.loc[p2.index[0], 'current_group'] = groupnum
            df.loc[p3.index[0], 'current_group'] = groupnum

        # create a new version of the overall df with the matches rows removed
        if i == 0:
            if i < number4groups*4:
                remaining = df.loc[df.index.difference((index, p2.index[0], p3.index[0], p4.index[0]))]
            else:
                remaining = df.loc[df.index.difference((index, p2.index[0], p3.index[0]))]
        if i > 0:
            if i < number4groups*4:
                remaining = remaining.loc[remaining.index.difference((index, p2.index[0], \
                                                                      p3.index[0], p4.index[0]))]
            else:
                remaining = remaining.loc[remaining.index.difference((index, p2.index[0], p3.index[0]))]

        if i == len(df) - 1:
            out = 1
            return out

        groupnum+=1

def create_match():
    # for all matches after the first
    global out, hr_full_data, hr_data, full_data, data
    counter = 0
    out = 0
    while out == 0:
        counter += 1
        out = perform_random_loop(data)  
        print(counter, '\r', end='')
        if counter >= 1000:
            print('match failed, not possible')
            break
            
    # use data to populate hr_data, full_data, and hr_full_data
    hr_data['group' + str(round_widget.value)] = data['current_group']
    hr_full_data = pd.merge(hr_full_data, hr_data, how='left')
    full_data.loc[data.index, :] = data[:]
    
    
#def create_match_download_file(data, pairing_round):
#    global match_file
#    match_file =  data[['first_name', 'last_name', 'email', 'BUID', 'department', 'current_group']]
#    match_file.rename(columns={'current_group':'round_%s_group'%pairing_round}, inplace=True)
#    return match_file

def create_download_link(df, title="Download CSV file", filename="data.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    display(HTML(html))
    
    
def send_emails(df, round_number):
    print('sending emails...', end='\r')
    for group_num in np.arange(1, df['current_group'].max()+1):
        group = df.loc[df['current_group'] == group_num]
        #group['email'].tolist()

        email_sender = 'bumeetup@bu.edu'
        email_password = 'SWsocial2022'

        email_recipients = group['email'].tolist()

        greeting = ', ' . join(group['first_name'].tolist()[:-1] + \
                               ['and ' + group['first_name'].tolist()[-1]])

        subject = 'BU Meetup Round %s'%str(round_number)
        body = \
'''Hello %s,

The %s of you have been successfully matched for Round %s of the BU Meetup! \
Reach out to each other and schedule a time to get coffee, go for a walk, have a meal, \
or anything else you’d like to do. 

Please be courteous to your group members: respond to each other and make every effort to meet in person. \
If you absolutely cannot find a time to meet in person, you may meet over Zoom. 

Feel free to reach out if you have any questions or concerns and we’ll do our best to get back to you. 

And most of all…have fun! 

~BU Meetup''' %(greeting, str(len(email_recipients)), str(round_number))

        em = EmailMessage()
        em['From'] = email_sender
        em['To'] = email_recipients
        em['Subject'] = subject
        em.set_content(body)

        context = ssl.create_default_context()

        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(email_sender, email_password)
            smtp.sendmail(email_sender, email_recipients, em.as_string())

    print('emails sent       ', '\r', end='')


style = {'description_width': 'initial'}

file_widget = widgets.Text(
    description='Participants file:', 
    disabled=False, 
    style=style)

datafile_widget = widgets.Text(
    description='Data file:', 
    disabled=False, 
    style=style)

round_widget = widgets.Dropdown(
    options=np.arange(1, 10),
    description='Matching round:', 
    disabled=False, 
    style=style)
import_button = widgets.Button(description='Import', style=style)
#possible_matches_botton = widgets.Button(description='Possible Matches', style=style)
create_match_button = widgets.Button(description='Create Match', style=style)
#write_match_button = widgets.Button(description='Write Match', style=style)
#complete_match_button = widgets.Button(description='Complete Match', style=style)
download_match_button = widgets.Button(description='Download Match', style=style)
download_data_button = widgets.Button(description='Download Data', style=style)
send_emails_button = widgets.Button(description='Send Emails', style=style)

def display_widget():
    display(round_widget), \
    display(file_widget), \
    display(datafile_widget), \
    display(import_button), \
#    display(possible_matches_botton), \
    display(create_match_button), \
#    display(write_match_button), \
    display(download_match_button), \
#    display(complete_match_button), \
    display(send_emails_button), \
    display(download_data_button)

data = None
number3groups = None
number4groups = None

def event_import(button):
    global hr_full_data, hr_data, full_data, data, number3groups, number4groups
    clear_output()
    display_widget()
    
    def convert_cell(cell):
        if cell != '[]':
            a = cell.strip('][').split(', ')
            b = [int(i) for i in a]
            return b
        else:
            return []

    # have to parse the inputs differently if it's the first round or subsequent rounds 
    if round_widget.value == 1:
        # hr_full_data is human-readable data with everyone who filled out the intake form
        hr_full_data = pd.read_excel(file_widget.value) 
        # hr_data is hr_full_data with only people opted into the current round  
        hr_data = hr_full_data[hr_full_data[hr_full_data.columns[-1]]=='Yes'].copy() 
        # full_data is [kind of] non-human-readable data for everyone 
        full_data = hr_full_data[['full_name', 'first_name', 'BUID', 'email', 'school', \
                                  'department']].copy()
        # data is [kind of] non-human-readable data for people opted into the round. 
        #     data is what I use to do all of the actual processing
        data = hr_data[['full_name','first_name','BUID','email','school','department']].copy()
       
        init(full_data)
        init(data)

    elif round_widget.value > 1:
        # hr_full_data is human-readable data with everyone who filled out the intake form
        hr_full_data = pd.read_excel(file_widget.value)
        # hr_data is hr_full_data with only people opted into the current round  
        hr_data = hr_full_data[hr_full_data[hr_full_data.columns[-1]]=='Yes'].copy() 
        # full_data is [kind of] non-human-readable data for everyone 
        full_data = pd.read_excel(datafile_widget.value, dtype=object)
        for index, row in full_data.iterrows():
            full_data.loc[index, 'prior_matches'] = convert_cell(row['prior_matches'])
    
        # data is [kind of] non-human-readable data for people opted into the round. 
        #     data is what I use to do all of the actual processing
        data = full_data[hr_full_data[hr_full_data.columns[-1]]=='Yes'].copy() 
    
    n = len(data)
    number3groups = n // 3
    number4groups = n - (number3groups * 3)

    print('imported participants')

def event_create_match(button):
    global hr_full_data, hr_data, full_data, data
    clear_output()
    display_widget()
    possible_matches()
    #print('round %i possible matches complete'%round_widget.value)
    create_match()
    if out == 1:
        print('round %i match created'%round_widget.value)
    if out == 0:
        print('round %i match failed'%round_widget.value)

def event_download_match(button):
    global hr_full_data, hr_data, full_data, data
    clear_output()
    display_widget()
    # the file to download is the human-readable full data
    return create_download_link(hr_full_data, title="download round %i match"%round_widget.value, \
                         filename="round_%i_match.csv"%round_widget.value)

def event_download_data(button):
    global hr_full_data, hr_data, full_data, data
    clear_output()
    display_widget()
    complete_match()
    full_data.loc[data.index, :] = data[:]
    return create_download_link(full_data, title="download round %i data"%round_widget.value, \
                         filename="round_%i_data.csv"%round_widget.value)

def event_send_emails(button):
    clear_output()
    display_widget()
    send_emails(data, round_widget.value)

import_button.on_click(event_import)
create_match_button.on_click(event_create_match)
#complete_match_button.on_click(event_complete_match)
download_match_button.on_click(event_download_match)
download_data_button.on_click(event_download_data)
send_emails_button.on_click(event_send_emails)

In [2]:
clear_output()
display_widget()

Dropdown(description='Matching round:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9), style=DescriptionStyle(descriptio…

Text(value='Fall2022_program/stress_test/Match1_intake_example1.xlsx', description='Participants file:', style…

Text(value='', description='Data file:', style=DescriptionStyle(description_width='initial'))

Button(description='Import', style=ButtonStyle())

Button(description='Create Match', style=ButtonStyle())

Button(description='Download Match', style=ButtonStyle())

Button(description='Send Emails', style=ButtonStyle())

Button(description='Download Data', style=ButtonStyle())